In [1]:
# Instal package
!pip install scikit-learn imbalanced-learn xgboost shap joblib

In [1]:
# Step 1: Load data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', 200)
path = "D:/LOAN APP PROJECT/data/cleaned_loan_approval_dataset.csv"  # change if needed
df2 = pd.read_csv(path)
print("Shape:", df2.shape)
df2.head()

Shape: (19999, 16)


,Loan_id,No_of_dependents,Education,Self_employed,Income_annum,Loan_amount,Loan_term,Cibil_score,Residential_assets_value,Commercial_assets_value,Luxury_assets_value,Bank_asset_value,Loan_status,Residential_assets_value_clean,Income_bin,Loan_bin
0,1,2,Graduate,No,9600000,29900000,12,778,2400000,17600000,22700000,8000000,Approved,2400000.0,High,High
1,2,0,Not Graduate,Yes,4100000,12200000,8,417,2700000,2200000,8800000,3300000,Rejected,2700000.0,Mid,Mid
2,3,3,Graduate,No,9100000,29700000,20,506,7100000,4500000,33300000,12800000,Rejected,7100000.0,High,High
3,4,3,Graduate,No,8200000,30700000,8,467,18200000,3300000,23300000,7900000,Rejected,18200000.0,High,High
4,5,5,Not Graduate,Yes,9800000,24200000,20,382,12400000,8200000,29400000,5000000,Rejected,12400000.0,High,High


In [2]:
df2.columns.to_list()

['Loan_id',
 'No_of_dependents',
 'Education',
 'Self_employed',
 'Income_annum',
 'Loan_amount',
 'Loan_term',
 'Cibil_score',
 'Residential_assets_value',
 'Commercial_assets_value',
 'Luxury_assets_value',
 'Bank_asset_value',
 'Loan_status',
 'Residential_assets_value_clean',
 'Income_bin',
 'Loan_bin']

In [3]:
# Print categorical columns with their unique categories
for col in df2.select_dtypes(include=['object', 'category']).columns:
    print(f"{col}: {df2[col].unique().tolist()}")

Education: ['Graduate', 'Not Graduate']
Self_employed: ['No', 'Yes']
Loan_status: [' Approved', ' Rejected']
Income_bin: ['High', 'Mid', 'Low']
Loan_bin: ['High', 'Mid', 'Low']


In [4]:
# Automatically clean all string/object columns
df2 = df2.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

In [5]:
# Print categorical columns with their unique categories
for col in df2.select_dtypes(include=['object', 'category']).columns:
    print(f"{col}: {df2[col].unique().tolist()}")

Education: ['Graduate', 'Not Graduate']
Self_employed: ['No', 'Yes']
Loan_status: ['Approved', 'Rejected']
Income_bin: ['High', 'Mid', 'Low']
Loan_bin: ['High', 'Mid', 'Low']


In [6]:
# convert all category in lower case
df2 = df2.apply(lambda x: x.str.lower() if x.dtype == "object" else x)
# Print categorical columns with their unique categories
for col in df2.select_dtypes(include=['object', 'category']).columns:
    print(f"{col}: {df2[col].unique().tolist()}")

Education: ['graduate', 'not graduate']
Self_employed: ['no', 'yes']
Loan_status: ['approved', 'rejected']
Income_bin: ['high', 'mid', 'low']
Loan_bin: ['high', 'mid', 'low']


In [7]:
# 🔽 Convert all column names to lowercase
df2.columns = df2.columns.str.lower()

In [8]:
df2.columns.to_list()

['loan_id',
 'no_of_dependents',
 'education',
 'self_employed',
 'income_annum',
 'loan_amount',
 'loan_term',
 'cibil_score',
 'residential_assets_value',
 'commercial_assets_value',
 'luxury_assets_value',
 'bank_asset_value',
 'loan_status',
 'residential_assets_value_clean',
 'income_bin',
 'loan_bin']

In [26]:
df2.to_csv("D:/LOAN APP PROJECT/data/cleaned_loan_approval_dataset.csv")

# STEP 2 : FEATURES ENGINEERING : 

In [9]:
# ================================
# FEATURE ENGINEERING V2 — UPDATED
# Uses your actual df columns list
# ================================
import numpy as np
import pandas as pd

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# --------- 0) Copy dataset ----------
df = df2.copy()
print("Starting FE V2. Input shape:", df.shape)
print("Columns available:", df.columns.tolist())

# --------- 1) Use cleaned residential assets if available ----------
resid_col = 'residential_assets_value_clean' if 'residential_assets_value_clean' in df.columns else 'residential_assets_value'
df[resid_col] = df[resid_col].fillna(0)

# --------- 2) Total assets (use cleaned residential assets) ----------
asset_cols = []
for c in ['residential_assets_value_clean', 'residential_assets_value', 'commercial_assets_value', 'luxury_assets_value', 'bank_asset_value']:
    if c in df.columns:
        asset_cols.append(c)
# pick the residential cleaned column if exists
if 'residential_assets_value_clean' in df.columns:
    # ensure we don't double count both residential_* columns
    asset_cols = [c for c in asset_cols if c != 'residential_assets_value'] if 'residential_assets_value' in asset_cols else asset_cols

for c in asset_cols:
    df[c] = pd.to_numeric(df[c], errors='coerce').fillna(0)

df['total_assets'] = df[asset_cols].sum(axis=1)

# --------- 3) Ensure numeric types for key columns ----------
to_numeric_cols = ['no_of_dependents', 'income_annum', 'loan_amount', 'loan_term', 'cibil_score']
for c in to_numeric_cols:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors='coerce')

# fill simple missing values for core numeric columns (will be refined in transformer)
for c in ['no_of_dependents', 'income_annum', 'loan_amount', 'loan_term', 'cibil_score']:
    if c in df.columns:
        df[c] = df[c].fillna(df[c].median())

# --------- 4) Create ratio / interaction style features ----------
# Use +1 or small epsilon to avoid div by zero
eps = 1e-9
if 'income_annum' in df.columns and 'loan_amount' in df.columns:
    df['income_to_loan'] = df['income_annum'] / (df['loan_amount'] + eps)
if 'total_assets' in df.columns and 'loan_amount' in df.columns:
    df['assets_to_loan'] = df['total_assets'] / (df['loan_amount'] + eps)
if 'income_annum' in df.columns and 'no_of_dependents' in df.columns:
    df['income_per_dependent'] = df['income_annum'] / (df['no_of_dependents'] + 1.0)
if 'loan_term' in df.columns and 'loan_amount' in df.columns:
    df['loan_amount_per_term'] = df['loan_amount'] / (df['loan_term'] + eps)

# Clip extreme outliers in created ratios (1st/99th percentiles)
for col in ['income_to_loan', 'assets_to_loan', 'income_per_dependent', 'loan_amount_per_term']:
    if col in df.columns:
        lower = df[col].quantile(0.01)
        upper = df[col].quantile(0.99)
        df[col] = df[col].clip(lower=lower, upper=upper)

# --------- 5) Drop identifiers / arbitrary binned columns ----------
drop_cols = ['loan_id', 'income_bin', 'loan_bin']
drop_these = [c for c in drop_cols if c in df.columns]
if drop_these:
    df = df.drop(columns=drop_these)
    print("Dropped columns:", drop_these)

# --------- 6) Prepare target encoding ----------
# Create loan_status_encoded if not exists
from sklearn.preprocessing import LabelEncoder
if 'loan_status_encoded' not in df.columns:
    if 'loan_status' not in df.columns:
        raise KeyError("Target 'loan_status' not found in df. Please check your dataset.")
    le = LabelEncoder()
    df['loan_status_encoded'] = le.fit_transform(df['loan_status'].astype(str))
    mapping = dict(zip(le.classes_, le.transform(le.classes_)))
    print("Created loan_status_encoded. Mapping:", mapping)
else:
    print("loan_status_encoded already present.")

# --------- 7) Select numeric and categorical columns for transformer ----------
# Exclude the target
exclude_cols = ['loan_status', 'loan_status_encoded']
cand_num = df.select_dtypes(include=[np.number]).columns.tolist()
numeric_cols = [c for c in cand_num if c not in exclude_cols]

# For categorical, keep these known categorical fields if present
possible_cat = ['education', 'self_employed']
categorical_cols = [c for c in possible_cat if c in df.columns]

print("Numeric columns BEFORE dropping correlated ones:", numeric_cols)
print("Categorical columns:", categorical_cols)

# --------- 8) Reduce multi-collinearity among numeric cols (drop one of very-highly-correlated pairs) ----------
if len(numeric_cols) > 1:
    # quick median impute for correlation calc
    tmp = df[numeric_cols].copy().fillna(df[numeric_cols].median())
    corr = tmp.corr().abs()
    upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))
    to_drop = [column for column in upper.columns if any(upper[column] > 0.90)]
    if to_drop:
        numeric_cols = [c for c in numeric_cols if c not in to_drop]
        print("Dropped (highly correlated) numeric cols:", to_drop)
    else:
        print("No highly correlated numeric columns to drop.")
else:
    print("Not enough numeric columns for correlation check.")

print("Numeric columns USED in transformer:", numeric_cols)

# --------- 9) Build preprocessing pipelines ----------
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('poly', PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)),
    ('scaler', StandardScaler())
])

# use sparse=False for compatibility across sklearn versions
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('cat', categorical_transformer, categorical_cols)
    ],
    remainder='drop'
)

# --------- 10) Fit-transform to get final features ----------
X = df.drop(columns=['loan_status', 'loan_status_encoded'], errors='ignore')
y = df['loan_status_encoded']

X_trans = preprocessor.fit_transform(X)

# --------- 11) Get feature names robustly (handles sklearn API differences) ----------
num_feature_names = []
cat_feature_names = []
try:
    # access poly step in pipeline to get names
    poly = preprocessor.named_transformers_['num'].named_steps['poly']
    # sklearn >=1.0 uses get_feature_names_out
    try:
        num_feature_names = poly.get_feature_names_out(numeric_cols)
    except Exception:
        num_feature_names = poly.get_feature_names(numeric_cols)
except Exception as e:
    # if no numeric cols or pipeline different, leave empty
    num_feature_names = []

try:
    ohe = preprocessor.named_transformers_['cat'].named_steps['onehot']
    try:
        cat_feature_names = ohe.get_feature_names_out(categorical_cols)
    except Exception:
        cat_feature_names = ohe.get_feature_names(categorical_cols)
except Exception:
    cat_feature_names = []

all_feature_names = list(num_feature_names) + list(cat_feature_names)
print("Transformed feature count:", len(all_feature_names))

# If transformer returns numpy array with shape and we couldn't get names, create generic names
if X_trans.shape[1] != len(all_feature_names):
    # fallback: create generic feature names
    all_feature_names = [f"f_{i}" for i in range(X_trans.shape[1])]
    print("Warning: couldn't retrieve exact feature names, using generic column names.")

df_final_features_v2 = pd.DataFrame(X_trans, columns=all_feature_names, index=df.index)
df_final_features_v2['loan_status_encoded'] = y.values

print("✅ df_final_features_v2 ready. Shape:", df_final_features_v2.shape)
print("Sample columns:", df_final_features_v2.columns[:20].tolist())

# Save optional copy (you can comment/uncomment)
# df_final_features_v2.to_csv('/mnt/data/df_final_features_v2.csv', index=False)

# End of FE V2


Starting FE V2. Input shape: (19999, 16)
Columns available: ['loan_id', 'no_of_dependents', 'education', 'self_employed', 'income_annum', 'loan_amount', 'loan_term', 'cibil_score', 'residential_assets_value', 'commercial_assets_value', 'luxury_assets_value', 'bank_asset_value', 'loan_status', 'residential_assets_value_clean', 'income_bin', 'loan_bin']
Dropped columns: ['loan_id', 'income_bin', 'loan_bin']
Created loan_status_encoded. Mapping: {'approved': np.int64(0), 'rejected': np.int64(1)}
Numeric columns BEFORE dropping correlated ones: ['no_of_dependents', 'income_annum', 'loan_amount', 'loan_term', 'cibil_score', 'residential_assets_value', 'commercial_assets_value', 'luxury_assets_value', 'bank_asset_value', 'residential_assets_value_clean', 'total_assets', 'income_to_loan', 'assets_to_loan', 'income_per_dependent', 'loan_amount_per_term']
Categorical columns: ['education', 'self_employed']
Dropped (highly correlated) numeric cols: ['residential_assets_value_clean', 'total_asset

In [23]:

df_final_features_v2.head(5)

,no_of_dependents,income_annum,loan_amount,loan_term,cibil_score,residential_assets_value,commercial_assets_value,luxury_assets_value,bank_asset_value,income_to_loan,assets_to_loan,income_per_dependent,loan_amount_per_term,no_of_dependents income_annum,no_of_dependents loan_amount,no_of_dependents loan_term,no_of_dependents cibil_score,no_of_dependents residential_assets_value,no_of_dependents commercial_assets_value,no_of_dependents luxury_assets_value,no_of_dependents bank_asset_value,no_of_dependents income_to_loan,no_of_dependents assets_to_loan,no_of_dependents income_per_dependent,no_of_dependents loan_amount_per_term,income_annum loan_amount,income_annum loan_term,income_annum cibil_score,income_annum residential_assets_value,income_annum commercial_assets_value,income_annum luxury_assets_value,income_annum bank_asset_value,income_annum income_to_loan,income_annum assets_to_loan,income_annum income_per_dependent,income_annum loan_amount_per_term,loan_amount loan_term,loan_amount cibil_score,loan_amount residential_assets_value,loan_amount commercial_assets_value,loan_amount luxury_assets_value,loan_amount bank_asset_value,loan_amount income_to_loan,loan_amount assets_to_loan,loan_amount income_per_dependent,loan_amount loan_amount_per_term,loan_term cibil_score,loan_term residential_assets_value,loan_term commercial_assets_value,loan_term luxury_assets_value,loan_term bank_asset_value,loan_term income_to_loan,loan_term assets_to_loan,loan_term income_per_dependent,loan_term loan_amount_per_term,cibil_score residential_assets_value,cibil_score commercial_assets_value,cibil_score luxury_assets_value,cibil_score bank_asset_value,cibil_score income_to_loan,cibil_score assets_to_loan,cibil_score income_per_dependent,cibil_score loan_amount_per_term,residential_assets_value commercial_assets_value,residential_assets_value luxury_assets_value,residential_assets_value bank_asset_value,residential_assets_value income_to_loan,residential_assets_value assets_to_loan,residential_assets_value income_per_dependent,residential_assets_value loan_amount_per_term,commercial_assets_value luxury_assets_value,commercial_assets_value bank_asset_value,commercial_assets_value income_to_loan,commercial_assets_value assets_to_loan,commercial_assets_value income_per_dependent,commercial_assets_value loan_amount_per_term,luxury_assets_value bank_asset_value,luxury_assets_value income_to_loan,luxury_assets_value assets_to_loan,luxury_assets_value income_per_dependent,luxury_assets_value loan_amount_per_term,bank_asset_value income_to_loan,bank_asset_value assets_to_loan,bank_asset_value income_per_dependent,bank_asset_value loan_amount_per_term,income_to_loan assets_to_loan,income_to_loan income_per_dependent,income_to_loan loan_amount_per_term,assets_to_loan income_per_dependent,assets_to_loan loan_amount_per_term,income_per_dependent loan_amount_per_term,education_graduate,education_not graduate,self_employed_no,self_employed_yes,loan_status_encoded
0,-0.300517,1.571669,1.629942,0.193435,1.042712,-0.783051,2.921737,0.838870,0.933933,-0.383589,-0.667454,0.545719,0.123771,0.511534,0.594299,-0.129777,0.049322,-0.601113,1.435580,0.207802,0.276847,-0.391747,-0.513162,1.404348,-0.057272,2.006085,1.280550,2.252023,-0.459488,3.659889,1.257665,1.362126,0.912469,0.606375,0.944303,0.437816,1.366621,2.318641,-0.411078,3.601536,1.254211,1.350501,1.572889,0.943438,1.021112,0.424396,0.697828,-0.581388,2.549821,0.756311,0.842668,-0.100801,-0.320288,0.552256,1.646572,-0.610929,3.834131,1.413998,1.499498,0.190863,-0.169204,0.961007,0.400706,-0.088507,-0.506653,-0.470825,-0.783868,-0.767271,-0.390476,-0.392568,2.549404,2.660934,2.308250,1.382869,2.089476,1.171522,0.790196,0.529881,0.134167,0.590958,0.201069,0.628210,0.235114,0.660831,0.247493,-0.507290,0.277491,0.057542,0.122145,-0.092833,0.187882,1.0,0.0,1.0,0.0,0
1,-1.483480,-0.377155,-0.304282,-0.504788,-1.032445,-0.736691,-0.682187,-0.669897,-0.513663,-0.287251,-0.936780,0.989929,-0.254918,-1.061668,-1.005597,-

In [11]:
df_final_features_v2.to_csv("df_final_features_v2.csv", index=False)

In [10]:
# print(df2.shape)
# df2.head(5)

(19999, 21)


,loan_id,no_of_dependents,education,self_employed,income_annum,loan_amount,loan_term,cibil_score,residential_assets_value,commercial_assets_value,luxury_assets_value,bank_asset_value,loan_status,residential_assets_value_clean,income_bin,loan_bin,total_assets,loan_to_income,loan_to_assets,debt_burden,cibil_bin
0,1,2,graduate,no,9600000,29900000,12,778,2400000,17600000,22700000,8000000,approved,2400000.0,high,high,50700000.0,3.114583,0.589744,0.495854,excellent
1,2,0,not graduate,yes,4100000,12200000,8,417,2700000,2200000,8800000,3300000,rejected,2700000.0,mid,mid,17000000.0,2.975610,0.717647,0.578199,very_low
2,3,3,graduate,no,9100000,29700000,20,506,7100000,4500000,33300000,12800000,rejected,7100000.0,high,high,57700000.0,3.263736,0.514731,0.444611,very_low
3,4,3,graduate,no,8200000,30700000,8,467,18200000,3300000,23300000,7900000,rejected,18200000.0,high,high,52700000.0,3.743902,0.582543,0.504105,very_low
4,5,5,not graduate,yes,9800000,24200000,20,382,12400000,8200000,29400000,5000000,rejected,12400000.0,high,high,55000000.0,2.469388,0.440000,0.373457,very_low


In [21]:
df2.shape

(19999, 16)

In [22]:
df2.head(5)

,loan_id,no_of_dependents,education,self_employed,income_annum,loan_amount,loan_term,cibil_score,residential_assets_value,commercial_assets_value,luxury_assets_value,bank_asset_value,loan_status,residential_assets_value_clean,income_bin,loan_bin
0,1,2,graduate,no,9600000,29900000,12,778,2400000,17600000,22700000,8000000,approved,2400000.0,high,high
1,2,0,not graduate,yes,4100000,12200000,8,417,2700000,2200000,8800000,3300000,rejected,2700000.0,mid,mid
2,3,3,graduate,no,9100000,29700000,20,506,7100000,4500000,33300000,12800000,rejected,7100000.0,high,high
3,4,3,graduate,no,8200000,30700000,8,467,18200000,3300000,23300000,7900000,rejected,18200000.0,high,high
4,5,5,not graduate,yes,9800000,24200000,20,382,12400000,8200000,29400000,5000000,rejected,12400000.0,high,high


In [54]:
df2.to_csv("riginal_dataset.csv")

In [16]:
# print(df_final_features['loan_status'].value_counts(normalize=True)*100)

loan_status
approved    62.79814
rejected    37.20186
Name: proportion, dtype: float64


In [19]:
df_final_features_v2.head(5)

,no_of_dependents,income_annum,loan_amount,loan_term,cibil_score,residential_assets_value,commercial_assets_value,luxury_assets_value,bank_asset_value,income_to_loan,assets_to_loan,income_per_dependent,loan_amount_per_term,no_of_dependents income_annum,no_of_dependents loan_amount,no_of_dependents loan_term,no_of_dependents cibil_score,no_of_dependents residential_assets_value,no_of_dependents commercial_assets_value,no_of_dependents luxury_assets_value,no_of_dependents bank_asset_value,no_of_dependents income_to_loan,no_of_dependents assets_to_loan,no_of_dependents income_per_dependent,no_of_dependents loan_amount_per_term,income_annum loan_amount,income_annum loan_term,income_annum cibil_score,income_annum residential_assets_value,income_annum commercial_assets_value,income_annum luxury_assets_value,income_annum bank_asset_value,income_annum income_to_loan,income_annum assets_to_loan,income_annum income_per_dependent,income_annum loan_amount_per_term,loan_amount loan_term,loan_amount cibil_score,loan_amount residential_assets_value,loan_amount commercial_assets_value,loan_amount luxury_assets_value,loan_amount bank_asset_value,loan_amount income_to_loan,loan_amount assets_to_loan,loan_amount income_per_dependent,loan_amount loan_amount_per_term,loan_term cibil_score,loan_term residential_assets_value,loan_term commercial_assets_value,loan_term luxury_assets_value,loan_term bank_asset_value,loan_term income_to_loan,loan_term assets_to_loan,loan_term income_per_dependent,loan_term loan_amount_per_term,cibil_score residential_assets_value,cibil_score commercial_assets_value,cibil_score luxury_assets_value,cibil_score bank_asset_value,cibil_score income_to_loan,cibil_score assets_to_loan,cibil_score income_per_dependent,cibil_score loan_amount_per_term,residential_assets_value commercial_assets_value,residential_assets_value luxury_assets_value,residential_assets_value bank_asset_value,residential_assets_value income_to_loan,residential_assets_value assets_to_loan,residential_assets_value income_per_dependent,residential_assets_value loan_amount_per_term,commercial_assets_value luxury_assets_value,commercial_assets_value bank_asset_value,commercial_assets_value income_to_loan,commercial_assets_value assets_to_loan,commercial_assets_value income_per_dependent,commercial_assets_value loan_amount_per_term,luxury_assets_value bank_asset_value,luxury_assets_value income_to_loan,luxury_assets_value assets_to_loan,luxury_assets_value income_per_dependent,luxury_assets_value loan_amount_per_term,bank_asset_value income_to_loan,bank_asset_value assets_to_loan,bank_asset_value income_per_dependent,bank_asset_value loan_amount_per_term,income_to_loan assets_to_loan,income_to_loan income_per_dependent,income_to_loan loan_amount_per_term,assets_to_loan income_per_dependent,assets_to_loan loan_amount_per_term,income_per_dependent loan_amount_per_term,education_graduate,education_not graduate,self_employed_no,self_employed_yes,loan_status_encoded
0,-0.300517,1.571669,1.629942,0.193435,1.042712,-0.783051,2.921737,0.838870,0.933933,-0.383589,-0.667454,0.545719,0.123771,0.511534,0.594299,-0.129777,0.049322,-0.601113,1.435580,0.207802,0.276847,-0.391747,-0.513162,1.404348,-0.057272,2.006085,1.280550,2.252023,-0.459488,3.659889,1.257665,1.362126,0.912469,0.606375,0.944303,0.437816,1.366621,2.318641,-0.411078,3.601536,1.254211,1.350501,1.572889,0.943438,1.021112,0.424396,0.697828,-0.581388,2.549821,0.756311,0.842668,-0.100801,-0.320288,0.552256,1.646572,-0.610929,3.834131,1.413998,1.499498,0.190863,-0.169204,0.961007,0.400706,-0.088507,-0.506653,-0.470825,-0.783868,-0.767271,-0.390476,-0.392568,2.549404,2.660934,2.308250,1.382869,2.089476,1.171522,0.790196,0.529881,0.134167,0.590958,0.201069,0.628210,0.235114,0.660831,0.247493,-0.507290,0.277491,0.057542,0.122145,-0.092833,0.187882,1.0,0.0,1.0,0.0,0
1,-1.483480,-0.377155,-0.304282,-0.504788,-1.032445,-0.736691,-0.682187,-0.669897,-0.513663,-0.287251,-0.936780,0.989929,-0.254918,-1.061668,-1.005597,-

In [20]:
df_final_features_v2.shape

(19999, 96)

# STEP - 3 : MODEL DEVELOPMENT

In [11]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import SelectFromModel

# Optional / try imports
try:
    from xgboost import XGBClassifier
except Exception:
    XGBClassifier = None
try:
    from lightgbm import LGBMClassifier
except Exception:
    LGBMClassifier = None
try:
    from imblearn.over_sampling import SMOTE
    smote_available = True
except Exception:
    SMOTE = None
    smote_available = False

# -------------------------
# 0) Basic checks & setup
# -------------------------
if 'df_final_features_v2' not in globals() and 'df_final_features_v2' not in locals():
    raise RuntimeError("df_final_features not found. Make sure to run the feature-engineering cells first.")

df = df_final_features_v2.copy()
print("Data shape:", df.shape)

# Ensure loan_status_encoded exists; otherwise encode
if 'loan_status_encoded' not in df.columns:
    le = LabelEncoder()
    df['loan_status_encoded'] = le.fit_transform(df['loan_status'])
    mapping = dict(zip(le.classes_, le.transform(le.classes_)))
else:
    # if both exist, inspect mapping by comparing unique values
    mapping = "loan_status_encoded exists already"

print("Label mapping (if computed):", mapping)
print("Target distribution:\n", df['loan_status_encoded'].value_counts(normalize=True).rename('prop'))

# -------------------------
# 1) Define features & target
# -------------------------
# If the notebook defined selected_features earlier use it; else use all except target columns
if 'selected_features' in globals() or 'selected_features' in locals():
    try:
        features = list(selected_features)
        features = [f for f in features if f in df.columns]
    except Exception:
        features = [c for c in df.columns if c not in ['loan_status','loan_status_encoded','loan_amount']]
else:
    features = [c for c in df.columns if c not in ['loan_status','loan_status_encoded','loan_amount']]

print(f"Using {len(features)} features for classification.")

X = df_final_features_v2[[
    # Demographics
    'no_of_dependents',

    # Income and loan
    'income_annum',
    'loan_amount',
    'loan_term',
    'income_per_dependent',

    # CIBIL and employment
    'cibil_score',
    'self_employed_yes',  # use one-hot encoded version
    'education_graduate',  # or 'education_not graduate', not both

    # Assets - pick main ones only, avoid combined or redundant versions
    'residential_assets_value',
    'commercial_assets_value',
    'luxury_assets_value',
    'bank_asset_value',

    # Ratios – choose only if strongly predictive (and avoid highly correlated ones)
    'income_to_loan',
    'assets_to_loan'
]]
y = df_final_features_v2['loan_status_encoded']


# Convert to numeric if any object types slipped in
X = X.apply(lambda col: pd.to_numeric(col, errors='coerce') if col.dtype == 'object' else col)
# Fill any remaining NA with median (safe fallback)
X = X.fillna(X.median())

# -------------------------
# 2) Train-test split
# -------------------------
RND = 42
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.20, random_state=RND)
print("Train shape:", X_train.shape, "Test shape:", X_test.shape)

# -------------------------
# 3) Optional SMOTE on training set
# -------------------------
use_smote = smote_available  # set to True/False depending on availability & preference
if use_smote:
    print("Applying SMOTE to training set (only).")
    sm = SMOTE(random_state=RND)  # Removed n_jobs argument
    X_train_res, y_train_res = sm.fit_resample(X_train, y_train)
    print("After SMOTE:", pd.Series(y_train_res).value_counts())
else:
    X_train_res, y_train_res = X_train, y_train

# -------------------------
# 4) Candidate models + param distributions (Randomized)
# -------------------------
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=RND)

models = []
param_distributions = []

# Logistic Regression (L1/L2)
models.append(('Logistic', LogisticRegression(solver='saga', max_iter=2000, random_state=RND)))
param_distributions.append({
    'C': [0.01, 0.1, 1, 5, 10],
    'penalty': ['l1', 'l2'],
    'class_weight': [None, 'balanced']
})

# Random Forest
models.append(('RandomForest', RandomForestClassifier(random_state=RND)))
param_distributions.append({
    'n_estimators': [100, 200, 400],
    'max_depth': [None, 6, 10, 20],
    'min_samples_split': [2, 4, 8],
    'class_weight': [None, 'balanced']
})


# -------------------------
# 5) RandomizedSearchCV loop
# -------------------------
from time import time
results = []
best_models = {}
n_iter_search = 20  # reduce/increase depending on compute

for (name, estimator), param_dist in zip(models, param_distributions):
    print("\n", "="*40)
    print("Training:", name)
    print("="*40)
    # Use RandomizedSearchCV
    rsearch = RandomizedSearchCV(
        estimator=estimator,
        param_distributions=param_dist,
        n_iter=min(n_iter_search, max(1, len(param_dist)*3)),
        scoring='accuracy',
        cv=skf,
        random_state=RND,
        n_jobs=-1,
        verbose=1
    )
    t0 = time()
    try:
        rsearch.fit(X_train_res, y_train_res)
    except Exception as e:
        print(f"  -> Failed to fit {name}: {e}")
        continue
    t1 = time()
    best = rsearch.best_estimator_
    best_params = rsearch.best_params_
    best_score_cv = rsearch.best_score_
    print(f"  Best CV accuracy for {name}: {best_score_cv:.4f} (time {t1-t0:.1f}s)")
    # Evaluate on test set
    y_pred = best.predict(X_test)
    try:
        y_prob = best.predict_proba(X_test)[:,1]
    except Exception:
        y_prob = None
    acc_test = accuracy_score(y_test, y_pred)
    roc = roc_auc_score(y_test, y_prob) if y_prob is not None else np.nan
    print(f"  Test accuracy for {name}: {acc_test:.4f}  ROC-AUC: {roc:.4f}")
    print("  Classification report (test):\n", classification_report(y_test, y_pred))
    print("  Confusion matrix:\n", confusion_matrix(y_test, y_pred))
    results.append({
        'model': name,
        'best_estimator': best,
        'best_params': best_params,
        'cv_accuracy': best_score_cv,
        'test_accuracy': acc_test,
        'roc_auc': roc
    })
    best_models[name] = best

# -------------------------
# 6) Summarize results and pick best by test_accuracy
# -------------------------
results_df = pd.DataFrame([{
    'model': r['model'],
    'cv_accuracy': r['cv_accuracy'],
    'test_accuracy': r['test_accuracy'],
    'roc_auc': r['roc_auc']
} for r in results]).sort_values(by='test_accuracy', ascending=False).reset_index(drop=True)

print("\n\n=== Summary of results ===")
print(results_df)

if not results:
    raise RuntimeError("No models were successfully trained.")

best_row = max(results, key=lambda x: x['test_accuracy'])
best_name = best_row['model']
best_model = best_row['best_estimator']
print(f"\nSelected best model: {best_name} with test accuracy {best_row['test_accuracy']:.4f}")

# -------------------------
# 7) Threshold tuning to maximize accuracy (optional for probability models)
# -------------------------
def tune_threshold(estimator, X_val, y_val):
    try:
        probs = estimator.predict_proba(X_val)[:,1]
    except Exception:
        return None, None
    thresholds = np.linspace(0,1,101)
    best_acc = -1
    best_thr = 0.5
    for thr in thresholds:
        preds = (probs >= thr).astype(int)
        acc = accuracy_score(y_val, preds)
        if acc > best_acc:
            best_acc = acc
            best_thr = thr
    return best_thr, best_acc

best_thr, best_thr_acc = tune_threshold(best_model, X_test, y_test)
if best_thr is not None:
    print(f"Optimal threshold for {best_name}: {best_thr:.2f} -> test accuracy {best_thr_acc:.4f}")
    # If improves, apply it and show confusion matrix
    if best_thr_acc > best_row['test_accuracy']:
        preds_thr = (best_model.predict_proba(X_test)[:,1] >= best_thr).astype(int)
        print("Confusion matrix with tuned threshold:\n", confusion_matrix(y_test, preds_thr))
else:
    print("Best model does not support predict_proba -> skipping threshold tuning.")

# -------------------------
# 8) Feature importances (if available)
# -------------------------
try:
    if hasattr(best_model, 'feature_importances_'):
        importances = best_model.feature_importances_
        feat_imp = pd.Series(importances, index=X.columns).sort_values(ascending=False)
        print("\nTop 20 feature importances:\n", feat_imp.head(20))
    elif hasattr(best_model, 'coef_'):
        coefs = best_model.coef_
        if coefs.ndim == 1:
            coef_series = pd.Series(coefs, index=X.columns).abs().sort_values(ascending=False)
            print("\nTop coefficients (abs):\n", coef_series.head(20))
except Exception as e:
    print("Could not obtain feature importances:", e)

# -------------------------
# 9) Save the best model
# -------------------------
out_path = "best_model_model1.pkl"
joblib.dump(best_model, out_path)
print("Saved best model to", out_path)

# -------------------------
# 10) Recommendations summary printout
# -------------------------
print("""
Recommendations (next steps) if accuracy is still low:
 - Check label quality (wrong labels), target leakage.
 - Try stacking/ensembling top models.
 - Try more feature engineering: interaction terms, polynomial features, more domain features.
 - Consider more aggressive hyperparameter search (increase n_iter or use Bayes optimization).
 - If class imbalance is severe, optimize for balanced accuracy or F1 instead.
 - Use cross-validation predictions (cross_val_predict) to detect overfitting/leakage.
""")


Data shape: (19999, 96)
Label mapping (if computed): loan_status_encoded exists already
Target distribution:
 loan_status_encoded
0    0.627981
1    0.372019
Name: prop, dtype: float64
Using 94 features for classification.
Train shape: (15999, 14) Test shape: (4000, 14)
Applying SMOTE to training set (only).
After SMOTE: loan_status_encoded
0    10047
1    10047
Name: count, dtype: int64

Training: Logistic
Fitting 5 folds for each of 9 candidates, totalling 45 fits
  Best CV accuracy for Logistic: 0.7474 (time 9.2s)
  Test accuracy for Logistic: 0.7348  ROC-AUC: 0.7968
  Classification report (test):
               precision    recall  f1-score   support

           0       0.84      0.71      0.77      2512
           1       0.61      0.77      0.68      1488

    accuracy                           0.73      4000
   macro avg       0.73      0.74      0.73      4000
weighted avg       0.76      0.73      0.74      4000

  Confusion matrix:
 [[1792  720]
 [ 341 1147]]

Training: Rand

In [12]:
trained_features = X_train.columns.tolist()
joblib.dump(trained_features, "trained_features.pkl")

['trained_features.pkl']

In [13]:
df2.head(20)

,loan_id,no_of_dependents,education,self_employed,income_annum,loan_amount,loan_term,cibil_score,residential_assets_value,commercial_assets_value,luxury_assets_value,bank_asset_value,loan_status,residential_assets_value_clean,income_bin,loan_bin
0,1,2,graduate,no,9600000,29900000,12,778,2400000,17600000,22700000,8000000,approved,2400000.0,high,high
1,2,0,not graduate,yes,4100000,12200000,8,417,2700000,2200000,8800000,3300000,rejected,2700000.0,mid,mid
2,3,3,graduate,no,9100000,29700000,20,506,7100000,4500000,33300000,12800000,rejected,7100000.0,high,high
3,4,3,graduate,no,8200000,30700000,8,467,18200000,3300000,23300000,7900000,rejected,18200000.0,high,high
4,5,5,not graduate,yes,9800000,24200000,20,382,12400000,8200000,29400000,5000000,rejected,12400000.0,high,high
5,6,0,graduate,yes,4800000,13500000,10,319,6800000,8300000,13700000,5100000,rejected,6800000.0,mid,mid
6,7,5,graduate,no,8700000,33000000,4,678,22500000,14800000,29200000,4300000,approved,22500000.0,high,high
7,8,2,graduate,yes,5700000,15000000,20,382,13200000,5700000,11800000,6000000,rejected,13200000.0,mid,mid
8,9,0,graduate,yes,800000,2200000,20,782,1300000,800000,2800000,600000,approved,1300000.0,low,low
9,10,5,not graduate,no,1100000,4300000,10,388,3200000,1400000,3300000,1600000,rejected,3200000.0,low,low


In [14]:
df_final_features_v2.head(2)

,no_of_dependents,income_annum,loan_amount,loan_term,cibil_score,residential_assets_value,commercial_assets_value,luxury_assets_value,bank_asset_value,income_to_loan,assets_to_loan,income_per_dependent,loan_amount_per_term,no_of_dependents income_annum,no_of_dependents loan_amount,no_of_dependents loan_term,no_of_dependents cibil_score,no_of_dependents residential_assets_value,no_of_dependents commercial_assets_value,no_of_dependents luxury_assets_value,no_of_dependents bank_asset_value,no_of_dependents income_to_loan,no_of_dependents assets_to_loan,no_of_dependents income_per_dependent,no_of_dependents loan_amount_per_term,income_annum loan_amount,income_annum loan_term,income_annum cibil_score,income_annum residential_assets_value,income_annum commercial_assets_value,income_annum luxury_assets_value,income_annum bank_asset_value,income_annum income_to_loan,income_annum assets_to_loan,income_annum income_per_dependent,income_annum loan_amount_per_term,loan_amount loan_term,loan_amount cibil_score,loan_amount residential_assets_value,loan_amount commercial_assets_value,loan_amount luxury_assets_value,loan_amount bank_asset_value,loan_amount income_to_loan,loan_amount assets_to_loan,loan_amount income_per_dependent,loan_amount loan_amount_per_term,loan_term cibil_score,loan_term residential_assets_value,loan_term commercial_assets_value,loan_term luxury_assets_value,loan_term bank_asset_value,loan_term income_to_loan,loan_term assets_to_loan,loan_term income_per_dependent,loan_term loan_amount_per_term,cibil_score residential_assets_value,cibil_score commercial_assets_value,cibil_score luxury_assets_value,cibil_score bank_asset_value,cibil_score income_to_loan,cibil_score assets_to_loan,cibil_score income_per_dependent,cibil_score loan_amount_per_term,residential_assets_value commercial_assets_value,residential_assets_value luxury_assets_value,residential_assets_value bank_asset_value,residential_assets_value income_to_loan,residential_assets_value assets_to_loan,residential_assets_value income_per_dependent,residential_assets_value loan_amount_per_term,commercial_assets_value luxury_assets_value,commercial_assets_value bank_asset_value,commercial_assets_value income_to_loan,commercial_assets_value assets_to_loan,commercial_assets_value income_per_dependent,commercial_assets_value loan_amount_per_term,luxury_assets_value bank_asset_value,luxury_assets_value income_to_loan,luxury_assets_value assets_to_loan,luxury_assets_value income_per_dependent,luxury_assets_value loan_amount_per_term,bank_asset_value income_to_loan,bank_asset_value assets_to_loan,bank_asset_value income_per_dependent,bank_asset_value loan_amount_per_term,income_to_loan assets_to_loan,income_to_loan income_per_dependent,income_to_loan loan_amount_per_term,assets_to_loan income_per_dependent,assets_to_loan loan_amount_per_term,income_per_dependent loan_amount_per_term,education_graduate,education_not graduate,self_employed_no,self_employed_yes,loan_status_encoded
0,-0.300517,1.571669,1.629942,0.193435,1.042712,-0.783051,2.921737,0.838870,0.933933,-0.383589,-0.667454,0.545719,0.123771,0.511534,0.594299,-0.129777,0.049322,-0.601113,1.435580,0.207802,0.276847,-0.391747,-0.513162,1.404348,-0.057272,2.006085,1.280550,2.252023,-0.459488,3.659889,1.257665,1.362126,0.912469,0.606375,0.944303,0.437816,1.366621,2.318641,-0.411078,3.601536,1.254211,1.350501,1.572889,0.943438,1.021112,0.424396,0.697828,-0.581388,2.549821,0.756311,0.842668,-0.100801,-0.320288,0.552256,1.646572,-0.610929,3.834131,1.413998,1.499498,0.190863,-0.169204,0.961007,0.400706,-0.088507,-0.506653,-0.470825,-0.783868,-0.767271,-0.390476,-0.392568,2.549404,2.660934,2.308250,1.382869,2.089476,1.171522,0.790196,0.529881,0.134167,0.590958,0.201069,0.628210,0.235114,0.660831,0.247493,-0.507290,0.277491,0.057542,0.122145,-0.092833,0.187882,1.0,0.0,1.0,0.0,0
1,-1.483480,-0.377155,-0.304282,-0.504788,-1.032445,-0.736691,-0.682187,-0.669897,-0.513663,-0.287251,-0.936780,0.989929,-0.254918,-1.061668,-1.005597,-

In [15]:
df2.shape

(19999, 16)

In [60]:
# Step 1: Fetch first 20 rows of preprocessed data
df_pred_input = df_final_features_v2.iloc[:20].copy()

# Step 2: Ensure feature column match
# Use only the features model was trained on
trained_feature_columns = best_model.feature_names_in_  # or manually store this during training
X_pred = df_pred_input[trained_feature_columns]

# Step 3: Fetch corresponding loan_ids from original df2
loan_ids = df2.iloc[:20]['loan_id'].values

# Step 4: Predict
y_pred = best_model.predict(X_pred)

# Step 5: Map numeric predictions to labels
label_map = {0: 'approved', 1: 'rejected'}
predicted_labels = [label_map.get(pred, 'unknown') for pred in y_pred]

# Step 6: Build and show result
result_df = pd.DataFrame({
    'loan_id': loan_ids,
    'loan_status': predicted_labels
})

print(result_df)


    loan_id loan_status
0         1    approved
1         2    rejected
2         3    rejected
3         4    rejected
4         5    rejected
5         6    rejected
6         7    approved
7         8    rejected
8         9    approved
9        10    rejected
10       11    approved
11       12    rejected
12       13    rejected
13       14    approved
14       15    rejected
15       16    approved
16       17    approved
17       18    approved
18       19    approved
19       20    approved


In [63]:
df.columns

Index(['no_of_dependents', 'income_annum', 'loan_amount', 'loan_term',
       'cibil_score', 'residential_assets_value', 'commercial_assets_value',
       'luxury_assets_value', 'bank_asset_value', 'income_to_loan',
       'assets_to_loan', 'income_per_dependent', 'loan_amount_per_term',
       'no_of_dependents income_annum', 'no_of_dependents loan_amount',
       'no_of_dependents loan_term', 'no_of_dependents cibil_score',
       'no_of_dependents residential_assets_value',
       'no_of_dependents commercial_assets_value',
       'no_of_dependents luxury_assets_value',
       'no_of_dependents bank_asset_value', 'no_of_dependents income_to_loan',
       'no_of_dependents assets_to_loan',
       'no_of_dependents income_per_dependent',
       'no_of_dependents loan_amount_per_term', 'income_annum loan_amount',
       'income_annum loan_term', 'income_annum cibil_score',
       'income_annum residential_assets_value',
       'income_annum commercial_assets_value',
       'income_annu

In [64]:
from sklearn.preprocessing import LabelEncoder

# Suppose these are the original labels you expect:
original_labels = ['approved', 'rejected']

le = LabelEncoder()
le.fit(original_labels)

print("Label encoding mapping:", dict(zip(le.classes_, le.transform(le.classes_))))


Label encoding mapping: {np.str_('approved'): np.int64(0), np.str_('rejected'): np.int64(1)}


In [65]:
print(df['loan_status_encoded'].value_counts())


loan_status_encoded
0    12559
1     7440
Name: count, dtype: int64


# STEP - 4 : MODEL EVALUATION : 

In [16]:
# ======================================
# 📦 MODEL 1 EVALUATION BLOCK (v2 aligned)
# ======================================

import joblib
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score

# -------------------------------
# 1️⃣ Load model + dataset
# -------------------------------
model_path = "best_model_model1.pkl"
model = joblib.load(model_path)
print("✅ Model loaded successfully.")

# Ensure df_final_features_v2 exists
df = df_final_features_v2.copy()

# -------------------------------
# 2️⃣ Define same features (exact order as used during training)
# -------------------------------
feature_order = [
    # Demographics
    'no_of_dependents',

    # Income and loan
    'income_annum',
    'loan_amount',
    'loan_term',
    'income_per_dependent',

    # CIBIL and employment
    'cibil_score',
    'self_employed_yes',
    'education_graduate',

    # Assets
    'residential_assets_value',
    'commercial_assets_value',
    'luxury_assets_value',
    'bank_asset_value',

    # Ratios
    'income_to_loan',
    'assets_to_loan'
]

# Filter and align columns
X = df[feature_order].copy()
y = df['loan_status_encoded']

# Convert to numeric and handle missing
X = X.apply(lambda col: pd.to_numeric(col, errors='coerce'))
X = X.fillna(X.median())

print("Dataset shape:", X.shape)

# -------------------------------
# 3️⃣ Prediction & Evaluation
# -------------------------------
y_pred = model.predict(X)

# Predict probabilities (if available)
try:
    y_prob = model.predict_proba(X)[:, 1]
except Exception:
    y_prob = None
    print("⚠️ Model does not support predict_proba, skipping ROC-AUC.")

# -------------------------------
# 4️⃣ Metrics
# -------------------------------
acc = accuracy_score(y, y_pred)
print(f"\n✅ Overall Accuracy: {acc:.4f}")

if y_prob is not None:
    roc_auc = roc_auc_score(y, y_prob)
    print(f"ROC-AUC Score: {roc_auc:.4f}")

print("\nClassification Report:")
print(classification_report(y, y_pred))

print("Confusion Matrix:")
print(confusion_matrix(y, y_pred))

# -------------------------------
# 5️⃣ Optional: threshold tuning on full data
# -------------------------------
def tune_threshold(estimator, X_val, y_val):
    try:
        probs = estimator.predict_proba(X_val)[:, 1]
    except Exception:
        return None, None
    thresholds = np.linspace(0, 1, 101)
    best_acc = -1
    best_thr = 0.5
    for thr in thresholds:
        preds = (probs >= thr).astype(int)
        acc = accuracy_score(y_val, preds)
        if acc > best_acc:
            best_acc = acc
            best_thr = thr
    return best_thr, best_acc

best_thr, best_thr_acc = tune_threshold(model, X, y)
if best_thr is not None:
    print(f"\n🔧 Optimal threshold: {best_thr:.2f} → Accuracy: {best_thr_acc:.4f}")
else:
    print("Threshold tuning skipped (no predict_proba).")

print("\n✅ Evaluation complete for Model 1.")


✅ Model loaded successfully.
Dataset shape: (19999, 14)

✅ Overall Accuracy: 0.9497
ROC-AUC Score: 0.9834

Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.96      0.96     12559
           1       0.93      0.94      0.93      7440

    accuracy                           0.95     19999
   macro avg       0.95      0.95      0.95     19999
weighted avg       0.95      0.95      0.95     19999

Confusion Matrix:
[[12002   557]
 [  448  6992]]

🔧 Optimal threshold: 0.51 → Accuracy: 0.9497

✅ Evaluation complete for Model 1.
